In [1]:

import csv
import random
import json

for pod in ['ap2', 'eu1', 'eu2', 'na1', 'na2']:
    id_to_version = {} # map each study/object id to respondingUiVersion
    nopatch_ids = set() # used to check for duplicate records in the nopatch results
    patch_ids = set() # used to check for duplicate records in the patch results
    with open("{}_2.csv".format(pod)) as nopatch, open("{}_1.csv".format(pod)) as patch:
        nopatch_reader = csv.reader(line.replace('\0', '')
                                    for line in nopatch)  # remove null bytes

        patch_reader = csv.reader(line.replace('\0', '')
                                  for line in patch)  # remove null bytes

        # generate id_to_version: start with nopatch. map v0 and v1 to v1, and map to v2 to v2.
        for i, line in enumerate(nopatch_reader):
            if i == 0:
                continue
            if line[1] in nopatch_ids:
                # skip duplicate nopatch: there won't be any
                continue
            else:
                version = line[3] if line[3] == '2' else (
                    '1' if line[3] == '1' or line[3] == '0' else '-1')
                id_to_version[line[1]] = version
                nopatch_ids.add(line[1])

        # generate id_to_version: continue with patch. map v0 and v1 to v1, and map to v2 to v2.
        for i, line in enumerate(patch_reader):
            if i == 0:
                continue
            if line[1] in patch_ids:
                # skip duplicate patch: the first one is the MOST RECENT one
                continue
            else:
                version = line[3] if line[3] == '2' else (
                    '1' if line[3] == '1' or line[3] == '0' else '-1')
                id_to_version[line[1]] = version
                patch_ids.add(line[1])

    # for i in range(0, 10):
    #     random_key = random.choice(list(id_to_version.keys()))
    #     print('entry: ')
    #     print(random_key, id_to_version[random_key])

    # id_to_version map is now complete, write queries to file
    with open("{}_idtoversion_queries.sql".format(pod), 'w') as queryfile:
        for key, value in id_to_version.items():
            queryfile.write(
                "update cpm.dbo.Activities SET RespondingUiVersion={} where ExternalActivityId='{}';".format(value, key))
            queryfile.write('\n')
